# Exam December 2022

In [1]:
import numpy as np
from scipy.stats import norm

N = norm.cdf

C:\Users\migue\AppData\Roaming\Python\Python38\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [34]:
# Data

# price of underlying at t=0
s0 = 60

# vol per annum of underlying
sigma = 0.12

# dividend
q = 0.12

# continuously compounded rf
r = 0.03

# time to maturity
T = 19
Tp = 19/12

# specific time mentioned throughout exam
t1 = 12
t1p = 12/12

dt = 1/12

# Q1.

Compute the price of a plain vanilla at-the-money European option with maturity at time 𝑇.

In [15]:
# Calculate price of call option w/ dividends

def bs_call_div(S_, X_, r_, Tp_, vol_, q_):
    # Calculate d1
    d1 = (np.log(S_/X_)+Tp_*((r_-q_)+(vol_**2)/2))/(vol_*np.sqrt(Tp_))
    # Calculate d2
    d2 = d1 - vol_ * np.sqrt(Tp_)
    # Calculate call value
    call = S_ * np.exp(-q_ * Tp_) * N(d1) - X_ * np.exp(-r_ * Tp_) * N(d2)
    '''
    print("S: ", S)
    print("K: ", X)
    print("r: ", r)
    print("T: ", T)
    print("sigma: ", var)
    print("d1: ", d1)
    print("d2: ", d2)
    print("N(d1): ",N(d1))
    print("N(d2): ",N(d2))
    '''
    return call

In [16]:
bs_call_div(S_ = s0, X_ = s0, r_ = r, Tp_ = Tp, vol_ = sigma, q_ = q)

0.7439486860690501

# Q2.

Compute the price of a European option with maturity at time 𝑇, and 𝐾 = 𝑆𝑡1

In [47]:
# With simulation

# Define function to construct price paths according to Ito's Lemma (Accounting for monthly dividend)

def price_paths_dividends(S0_, r_, sigma_, T_, dt_, n_paths, q_):
    paths = []
    for iteration in np.arange(0,n_paths,1):
        current_price = S0_
        dividend = S0_ * q_
        iteration_path = [current_price]
        z = norm.rvs(size = T_)
        for step in np.arange(1, T_, 1):
            if (step+1) % 6 == 0:
                current_price = (current_price * np.exp((r_ - ((sigma_**2)/2)) * dt_ + z[step-1] * sigma_ * np.sqrt(dt_))) - dividend
                iteration_path.append(current_price)
            else:
                current_price = current_price * np.exp((r_ - ((sigma_**2)/2)) * dt_ + z[step-1] * sigma_ * np.sqrt(dt_))
                iteration_path.append(current_price)
        paths.append([iteration, iteration_path])
    return paths

In [48]:
# Search for average price at time T

def average_price(paths_, time):
    sum = 0
    for path_ in range(0, len(paths_)):
        sum += paths_[path_][1][time-1]
    return sum/len(paths_)

In [49]:
# Construct price paths (adjustable number of iterations)

iter = 2000
paths = price_paths_dividends(S0_ = s0, r_ = r, sigma_ = sigma, T_ = T, dt_ = dt, n_paths = iter, q_ = q)

# Check average price at maturity
average_price(paths, T)


40.44843486713034

In [50]:
# Define function to calculate call payoff

def call_payoff(sT_, K_):
    return max(sT_ - K_, 0)

In [51]:
# function to simulate payoffs
def simulate_payoffs(paths_, t_, T_):
    payoff_list = []
    for path in paths_:
        sT = path[1][T_-1]
        st1 = path[1][t_-1]
        pi = call_payoff(sT, st1)
        payoff_list.append(pi)
    return payoff_list

In [52]:
discount_payoffs = np.exp(-r*Tp)

q2 = np.average(simulate_payoffs(paths, t1, T))
print("Average Call value with expiration at T and K = St1: ", q2*discount_payoffs)

Average Call value with expiration at T and K = St1:  0.20958631948316303


# 3.

Compute the price of a European option with maturity at time 𝑇, and 𝐾 = 𝑆𝑎𝑣𝑔, whereas 𝑆𝑎𝑣𝑔 represents the geometric average of the underlying asset’s price during the life of the option.

In [25]:
from scipy.stats import gmean

# simulate payoffs w/ geometric average
def simulate_payoffs_gavg(paths_, T_):
    payoff_list = []
    for path in paths_:
        k = gmean(path[1])
        sT = path[1][T_-1]
        pi = call_payoff(sT, k)
        payoff_list.append(pi)
    return payoff_list

In [26]:
q3 = np.average(simulate_payoffs_gavg(paths, T))
print("Average Call value with expiration at T and K = geo_avg: ", q3*discount_payoffs)

Average Call value with expiration at T and K = geo_avg:  0.4763951506716971


# 4.

Compute the price of a European option with maturity at time 𝑇, where at time 𝑡1 you must choose
either 𝐾 = 𝑆0 or 𝐾 = 𝑎𝑣𝑒𝑟𝑎𝑔𝑒{𝑆𝑡1, … , 𝑆𝑇}, where 𝑎𝑣𝑒𝑟𝑎𝑔𝑒{… } stands for the geometric average.

In [29]:
from scipy.stats import gmean

# simulate payoffs of chooser option
def simulate_payoffs_chooser(paths_, T_):
    payoff_list = []
    for path in paths_:
        k1 = gmean(path[1])
        k2 = path[1][0]
        sT = path[1][T_-1]
        if k1 < k2:
            pi = call_payoff(sT, k1)
            payoff_list.append(pi)
        else:
            pi = call_payoff(sT, k2)
            payoff_list.append(pi)
    return payoff_list

In [30]:
q4 = np.average(simulate_payoffs_chooser(paths, T))
print("Average Call value with choice: ", q4*discount_payoffs)

Average Call value with choice:  10.513256965167482


# 5.

Suppose that the option described in question 4 also had an up-and-out characteristic (if you are
analysing a call) or a down-and-out characteristic (if you are analysing a put). Further suppose that the barrier would be defined at 𝑡1 and would be equal to 1.15 ∗ 𝑆𝑡1 (if you are analysing a call) or 0.85 ∗ 𝑆𝑡1 (if you are analysing a put). And further assume that the frequency with which we would observe whether the price has crossed the barrier is daily, between 𝑡1 and 𝑇.

What is the risk-neutral probability that the barrier will be breached? And how would this feature affect the option’s fair value?

In [45]:
# count barrier breaches as % of paths
def count_breach(paths_, t1_, T_):
    count_breach = 0
    for path in paths_:
        found = 0
        for value in path[1][t1_-1:T_-1]:
            if value > (path[1][t1_-1] * 1.15):
                found = 1
        if found > 0:
            count_breach += 1
    return count_breach / len(paths_)

In [46]:
count_breach(paths, t1, T)

0.332

In [51]:
# function to signal breaches
def signal_breaches(paths_, t1_, T_):
    breach_list = []
    for npath in range(0, len(paths_)):
        path = paths[npath]
        found = 0
        for value in path[1][t1_-1:T_-1]:
            if value > (path[1][t1_-1] * 1.15):
                found = 1
        if found > 0:
            breach_list.append(npath)
    return breach_list

In [53]:
from scipy.stats import gmean

# function to simulate payoffs of option
def simulate_payoffs_chooser_barrier(paths_, t1_, T_):
    payoff_list = []
    breaches = signal_breaches(paths_, t1_, T_)
    for npath in range(0, len(paths_)):
        if npath not in breaches:    
            path = paths[npath]
            k1 = gmean(path[1])
            k2 = path[1][0]
            sT = path[1][T_-1]
            if k1 < k2:
                pi = call_payoff(sT, k1)
                payoff_list.append(pi)
            else:
                pi = call_payoff(sT, k2)
                payoff_list.append(pi)
        else:
            payoff_list.append(0)
    return payoff_list

In [55]:
q5 = np.average(simulate_payoffs_chooser_barrier(paths, t1, T))
print("Average Call value with choice and barrier: ", q5*discount_payoffs)

Average Call value with choice and barrier:  3.5533647940625266
